In [ ]:
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np
import os

#Gets the path of current directory
dir=os.path.realpath('.')
#Lists files in current directory, excluding child directories
list_files=list(os.walk(dir))[0][2]
#Selects final_data csv file name
final_data_file_name=[file for file in list_files if 'final_data' in file][0]
if len(final_data_file_name)==0:
    print('There is not final data file')
else:
    #Generates dataframe from file
    df=pd.read_csv(final_data_file_name)

Proportion of null values for each column

In [ ]:
print('df.shape:',df.shape)
round(
(df.isna().sum()/df.shape[0])
.sort_values(ascending=False)
,2
)

Number of properties published by date

In [ ]:
prop_per_date=df['date'].value_counts().sort_index()[1:]
week=['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

#This eliminates some outliers due to periods when there was no data extraction
prop_per_date=prop_per_date[prop_per_date<100]

#Creates a dictionary with days of the week instead of dates
change_index=(
    #Dictionary comprehension to change dates to weekdays 
    {str(pd.to_datetime(date).date())
    :str(week[pd.to_datetime(date).weekday()])+' '+str(pd.to_datetime(date).day)
 for date in prop_per_date.index}
)
#Define 'red' for weekend days and 'blue' for weekdays as colorbar
bar_colors=['r' if ('Sat' in day or 'Sun' in day) 
else 'b' for day in change_index.values()]

(
prop_per_date
.rename(index=change_index)
.plot(kind='bar',figsize=(15,3),color=bar_colors)
)
plt.xticks(np.arange(0,len(prop_per_date),10))
plt.show()



In [ ]:
#Filtering days with high number of properties due to previous missing files
prop_per_date[prop_per_date<50].sort_index().rolling(30).mean().plot()
plt.xticks(rotation=90)
plt.show()

Distribution by price

In [ ]:
sns.histplot(data=df,x='price',fill=True,alpha=0.2,kde=True,bins=np.arange(0,5000,200))
plt.xticks(np.arange(0,4000,500),rotation=90)
plt.show()

In [ ]:
df['price']=pd.to_numeric(df['price'],errors='coerce')

In [ ]:
df.query(f'bedrooms==1 & bathrooms==1')[['price','date']].groupby('date').median().rolling(30).mean().plot()
plt.xticks(rotation=90)
plt.show()

Agents market share by number of properties published

In [ ]:
(
    df['agent_name']
    .value_counts(sort=True,ascending=False)
    .head(20)
    .plot(kind='bar')
)
plt.show()

Distribution by number of bedrooms and bathrooms

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(10,2))

df_plot={}
for num,room in [(0,'bathrooms'),(1,'bedrooms')]:
        df_plot[room]=df.query('bathrooms<=2 & bedrooms <=3')[room].value_counts().sort_index()
        ax[num].bar(height=df_plot[room].values,x=df_plot[room].index)
        ax[num].set_xlabel(f'Number of {room}')
        ax[num].set_ylabel('Count')
        ax[num].set_xticks(list(range(1,4)))
        ax[num].set_yticks(list(range(0,df_plot[room].values.max(),500)))
plt.show()

Distribution by property type

In [ ]:
prop_type=df['property_type'].value_counts().sort_values(ascending=False)
fig,ax=plt.subplots()
ax.bar(height=prop_type.values,x=prop_type.index)
ax.set_xlabel('property type')
ax.set_ylabel('Count')
ax.set_xticklabels(prop_type.index,rotation=90)
plt.show()

Proportion of properties that are not flats

In [ ]:
flats=(df[df['property_type']=='Flat']['property_type'].value_counts().sum()/df.shape[0])
print('Percentaje of flats:',round(flats*100,1),'%')

 Changing values 'Now' and 'Ask Agent' from let_available_date' column (Part of this should be in the cleaning section)

In [ ]:
now_index=df[df['let_available_date']=='Now'].index
df.loc[now_index,'let_available_date']=df.loc[now_index,'date']

ask_agent_index=df[df['let_available_date']=='Ask agent'].index
df.loc[ask_agent_index,'let_available_date']=pd.NA

df['let_available_date']=pd.to_datetime(df['let_available_date'],
                                        errors='coerce',
                                        format='%d/%m/%Y').dt.date



Converting 'date' columns to datetime (This also should be in the cleaning section)

In [ ]:
df['date']=df['date'].astype('datetime64[ns]').dt.date

Distribution of days properties last published in the market

In [ ]:
df['days_on_market'].value_counts().sort_index().plot(kind='bar',figsize=(10,3),xticks=range(0,140,10))

Analysis acording if they have min tenancy period

In [ ]:
df['min_tenancy'].value_counts().sort_values(ascending=False)

Short term or long term rent

In [ ]:
df['let_type'].value_counts()

Furnished or unfurnished property

In [ ]:
round(
df['furnish_type'].value_counts()/df.shape[0]*100
,2
)

Bivariant analysis- Prices vs number of bedrooms

In [ ]:
sns.histplot(data=df[(df['bedrooms']<3) & (df['bathrooms']==1)],
            x='price',
            hue='bedrooms',
            palette=['red','green'],
            fill=True,
            alpha=0.5,
            kde=True,
            bins=30
            )
#Plot formatting
plt.xlim(0,2500)
plt.xticks(list(range(0,2500,200)))
plt.title('Properties with 1 bathroom')
plt.show()

sns.histplot(data=df[(df['bedrooms']==2) & (df['bathrooms']==2)],
            x='price',
            hue='bedrooms',
            fill=True,
            palette=['red'],
            alpha=0.5,
            kde=True,
            bins=30,
            )
plt.xlim(0,2500)
plt.title('Properties with 2 bathrooms')
plt.show()

Bivariant analysis- Prices vs number of bedrooms (outliers excluded)

In [ ]:
room1_q1,room1_q3=df[(df['bedrooms']==1) & (df['bathrooms']==1)]['price'].quantile(q=[0.25,0.75])
room2_q1,room2_q3=df[(df['bedrooms']==2) & (df['bathrooms']==1)]['price'].quantile(q=[0.25,0.75])

room1_lim_inf=room1_q1-(room1_q3-room1_q1)*1.5
room1_lim_sup=room1_q3+(room1_q3-room1_q1)*1.5

room2_lim_inf=room2_q1-(room2_q3-room2_q1)*1.5
room2_lim_sup=room2_q3+(room2_q3-room2_q1)*1.5

data_1_room=df[(df['bedrooms']==1) &
                (df['price']>room1_lim_inf) &
               (df['price']<=1200) &
                (df['bathrooms']==1)]

data_2_room=df[(df['bedrooms']==2) &
                (df['price']>room2_lim_inf) &
                (df['price']<room2_lim_sup) &
                (df['bathrooms']==1)]

sns.histplot(data=data_1_room,
            x='price',
            fill=True,
            alpha=0.5,
            kde=True,
            bins=np.arange(400,2000,50),
            #stat='probability'
            )

sns.histplot(data=data_2_room,
            x='price',
            fill=True,
            alpha=0.5,
            kde=True,
            bins=np.arange(400,2000,50),
            #stat='probability'
            )

plt.legend(['1 bedroom','2 bedrooms'])
plt.title('Price distribution for properties with one bathroom. Outliers excluded')
plt.show()


In [ ]:
sns.catplot(data=df.query('bathrooms==1 &\
                        bedrooms<=2 &\
                        furnish_type in ["Furnished", "Unfurnished"]'),
            y='price',x='furnish_type',col='bedrooms',kind='box')




Normal disribution fit for properties with one bathroom and one bedroom

In [ ]:
#property prices for 1 bedroom flat adjust to a norrmal distribution for flats below 1200 pcm.
#There is the need to find out if properties above 1200pcm are part of this population or they 
#are part of a different one.
sns.kdeplot(data=data_1_room,
            x='price',
            fill=True,
            alpha=0.5,
            common_norm=True,
            bw_adjust=0.6
            )

mu,std=norm.fit(data_1_room['price'])

x=np.linspace(400,2500)
normal_dist=norm.pdf(x,mu,std)
plt.plot(x,normal_dist,'r',linewidth=2,linestyle='dashed')
plt.show()


Normal disribution fit for properties with one bathroom and two bedrooms

In [ ]:
sns.kdeplot(data=data_2_room,
            x='price',
            fill=True,
            alpha=0.5,
            common_norm=True,
            bw_adjust=1
            )

mu,std=norm.fit(data_2_room['price'])

x=np.linspace(400,2500)
normal_dist=norm.pdf(x,mu,std)
plt.plot(x,normal_dist,'r',linewidth=2,linestyle='dashed')
plt.show()

Price distribution for each one of the 10 main agents

In [ ]:
main_agents=(df['agent_name']
                    .value_counts(sort=True,ascending=False)
                    .head(10)
                    )
main_agents_index=main_agents.index.tolist()
                    
df_main_agents=df[df['agent_name'].isin(main_agents_index)]

sns.boxplot(data=df_main_agents.query('bedrooms<3'),
                x='agent_name',
                y='price',
                hue='bedrooms')
#sns.stripplot(data=df_main_agents.query('bedrooms<3'),
#                x='agent_name',
#                y='price',
#                hue='bedrooms',
#                dodge=True)

plt.xticks(rotation=90)
plt.show()

Price distribution by furnish type, number of bathrooms and number of bedrooms

In [ ]:
classification=df[(df['bedrooms']<=2) &
                (df['bathrooms']<=2) &
            (df['furnish_type'].isin(['Furnished','Unfurnished']))
]
round(
classification.groupby(['bathrooms','bedrooms','furnish_type'])['price']
.agg(['median','mean','std','count'])
,0
)
sns.displot(data=classification,
            x="price",
            col="bedrooms",
            row="bathrooms",
            hue="furnish_type",
            height=5,
            aspect=1.5
            )

Analysis days_on_market vs price vs number of bedrooms

In [ ]:

#for i in range(1,5):
#    plt.figure(figsize=(7,1))
#    plt.ylim(0,60)
#    (
#    df[df['bedrooms']==i]['days_on_market']
#    .value_counts()
#    .sort_index()
#    .plot(kind='bar')
#    )
#    
#    plt.title('days on the market - {} bedrooms'.format(i))
#    plt.show()
    
data=df.query('bedrooms<=5')
plt.figure(figsize=(5,5))
sns.boxplot(data=data,y='days_on_market',x='bedrooms')
sns.stripplot(data=data,y='days_on_market',x='bedrooms')
plt.show()

Analysis EPC vs price vs number of bedrooms

In [ ]:
EPC=(
df[
(df['EPC_rating'].isin(['A','B','C','D','E','F','G']))
& 
(df['bedrooms']<=4)
]
[['EPC_rating','price','bedrooms']]
.sort_index()
)


#there is no influence of the EPC in the price of the properties
sns.stripplot(data=EPC,y='price',x='EPC_rating',hue='bedrooms',dodge=True,alpha=0.25,order=['A','B','C','D','E','F','G'])
sns.boxplot(data=EPC,y='price',x='EPC_rating',hue='bedrooms',order=['A','B','C','D','E','F','G'])


In [ ]:
council_tax=(
df[(df['bathrooms']==1) & (df['bedrooms']<4) ][['council_tax_band','price','bedrooms']]
.sort_values('council_tax_band')
)


#there is no influence of the EPC in the price of the properties
sns.stripplot(data=council_tax,y='price',x='council_tax_band',color='red',alpha=0.5,hue='bedrooms',dodge=True)
sns.boxplot(data=council_tax,y='price',x='council_tax_band',hue='bedrooms')

plt.show()


In [ ]:
data=df[
(df['bathrooms']==1)
& 
(df['bedrooms']==1)
]
round(
    data.groupby(['bathrooms','bedrooms','postcode_district','council_tax_band','furnish_type'])['price']
    .agg(['median','mean','std','count'])
    .sort_values('count',ascending=False)
    ,0
)


In [ ]:
data=df[
(df['bathrooms']==1)
& 
(df['bedrooms']==1)
&
(df['postcode_district']=='EH{}'.format(11))
# &
# (df['council_tax_band']=='B')
&
(df['furnish_type']=='Furnished')

]
sns.histplot(data=data,x='price',bins=np.arange(700,1300,25))


In [ ]:
(df.groupby('postcode_district')
    .count()
    ['url']
    .sort_values(ascending=False)
    .plot(kind='bar',figsize=(5,2))
)

In [ ]:
#Trying to find out if there is a time pattern in property
# publications based on postcode district
for i in range(1,13): 
    (df.groupby(['postcode_district','date'])
    .count()
    ['url']
    .reset_index()
    .rename(columns={'url':'num_properties'})
    .query('postcode_district=="EH{}"'.format(i))
    .set_index('date')
    [1:]
    .plot(kind='bar',figsize=(12,3),title='EH{}'.format(i))
    )